# Aim: To understand how to use FB_Prophet
* We won't be able to critically analyse the reason for price fall or rise during certain time of year. 
                Reason Being
                1. We don't know from where these Avocardos are coming
                2. Thus don't know about its harvesting time.
                3. Though we have voume of avocardo in transaction. Lets see how we can get that in our favour
* **And remember Prophet works best with SEASONAL DATA**
* This is why I think predicting price of Avocado will be best fit as supply of Avocado is dependent on season of harvesting.

In [ ]:
# Installing Prophet

! pip install prophet

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random

# Importing Prophet
from fbprophet import Prophet

# importing module
import warnings
  
# adding entry into the specifications
# of the warnings filter.
warnings.filterwarnings('ignore')

In [ ]:
df= pd.read_csv('../input/avocado-prices/avocado.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

## EDA

In [ ]:
sns.heatmap(df.isnull(), yticklabels= False,cbar= False , cmap='viridis' )
# there is no null value

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from plotly import tools

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

In [ ]:
# we saw there are 2 type of Avocardo Lets check their percentage

Type=df.groupby('type')['Total Volume'].agg('sum')

values=[Type['conventional'],Type['organic']]
labels=['conventional','organic']

trace=go.Pie(labels=labels,values=values)
py.iplot([trace])

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(x='Total Volume', y = 'AveragePrice', hue= 'type', data= df)

In [ ]:
con=df[df['type']=='conventional'].groupby('year')['Total Volume'].agg('mean')
org=df[df['type']=='organic'].groupby('year')['Total Volume'].agg('mean')

trace1=go.Bar(x=con.index,y=con,name="Conventional",
             marker=dict(
        color='rgb(58,200,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.7)

trace2=go.Bar(x=con.index,y=org,name="Organic",
             marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5),
        ),
    opacity=0.7)

data=[trace1,trace2]

layout=go.Layout(barmode="stack",title="Organic vs. Conventional (Mean Volume)",
                yaxis=dict(title="Volume"))
fig=go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
import seaborn as sns
# Split Date into 3 different columns

df['Year'], df['Month'],  df['Day'] = df['Date'].str.split('-').str

# compare month wise price of avocado 
# organic and conventional 

df['Month'] = df['Month'].replace({'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', 
                                   '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October', 
                                   '11': 'November', '12': 'December'})


ax = sns.catplot(x="Month", y="AveragePrice", hue="type", 
            kind="box", data=df, height=8.5, linewidth=2.5, aspect=2.8,palette="Set2");

* We can see only 2.81% of all Avocado sold is organic
* Thus we can conclude the price of Organic Avocado won't affect the price over all 
* So it might be better if we drop it

In [ ]:
sns.pairplot(df.iloc[:,8:11], palette="husl",height=5.5)

In [ ]:
df.region.unique().sum()

In [ ]:
# Sort by date
df = df.sort_values('Date')
df

In [ ]:
# Date vs average price

plt.figure(figsize=(20,10))
plt.xticks(np.arange(df.shape[0])[::5], df.Date[::5], rotation = 45)
plt.plot(df['Date'], df['AveragePrice'])
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout()
plt.savefig('timeseries.png')
# uncomment to get grid plt.grid()


## Setting Axis-Level Tick Frequency in Matplotlib
You can use the set_xticks() and set_yticks() functions on the returned Axes instance when adding subplots to a Figure. Let's create a Figure with two axes and change the tick frequency on them separately:

In [ ]:
''' #converting dataframe to array
date = pd.Series(df['Date'])
x=type(date.to_numpy())

# Age price
av= pd.Series(df['AveragePrice'])
y=type(date.to_numpy())

print(y)'''

In [ ]:
'''#converting dataframe to array
date = pd.Series(df['Date'])
x=type(date.to_numpy())

# Age price
av= pd.Series(df['AveragePrice'])
y=type(date.to_numpy())
                 

plt.figure(figsize=(20,10))
plt.xticks(rotation = 90)
plt.xticks(np.arange(0, len(x)+1, 5))
plt.plot(x, y)
plt.tick_params(axis='x', which='major', labelsize=10)
plt.tight_layout()
# uncomment to get grid plt.grid()'''


In [ ]:
# correlation version region and price
plt.figure(figsize=[20,15])
plot= sns.countplot(x= 'region', data= df)
plt.xticks(rotation = 90)
# plt.setp(plot.get_xticklabels(),rotation=90)
# Was returning lots of None, so decided to go with
# Didn't work plt.xticks(rotation = 90, np.array(0,4,step=5))


In [ ]:
region_list=list(df.region.unique())
average_price=[]

for i in region_list:
    x=df[df.region==i]
    region_average=sum(x.AveragePrice)/len(x)
    average_price.append(region_average)

df1=pd.DataFrame({'region_list':region_list,'average_price':average_price})
new_index=df1.average_price.sort_values(ascending=False).index.values
sorted_data=df1.reindex(new_index)

plt.figure(figsize=(24,10))
ax=sns.barplot(x=sorted_data.region_list,y=sorted_data.average_price)

plt.xticks(rotation=90)
plt.xlabel('Region')
plt.ylabel('Average Price')
plt.title('Average Price of Avocado According to Region')

In [ ]:
filter1=df.region!='TotalUS'
df1=df[filter1]

region_list=list(df1.region.unique())
average_total_volume=[]

for i in region_list:
    x=df1[df1.region==i]
    average_total_volume.append(sum(x['Total Volume'])/len(x))
df3=pd.DataFrame({'region_list':region_list,'average_total_volume':average_total_volume})

new_index=df3.average_total_volume.sort_values(ascending=False).index.values
sorted_data1=df3.reindex(new_index)

plt.figure(figsize=(22,10))
ax=sns.barplot(x=sorted_data1.region_list,y=sorted_data1.average_total_volume)

plt.xticks(rotation=90)
plt.xlabel('Region')
plt.ylabel('Average of Total Volume')
plt.title('Average of Total Volume According to Region')

In [ ]:
sns.countplot(x= 'year', data= df)

In [ ]:
# Since Prophet require only 2 columns namely
# Data and Avg Price
# Lets create new df for prophet only

prophet_df= df[['Date', 'AveragePrice']]
prophet_df

In [ ]:
# Prophet expect Date in its variable as DS and target variable as Y to operate. This how it was designed.
# Thus have to rename our column names in prophet_df
prophet_df= prophet_df.rename(columns={'Date' :'ds' , 'AveragePrice':'y'})
prophet_df

In [ ]:
prophet_df.info()

In [ ]:
prophet_df.describe()

## Make Predictions

In [ ]:
# Intantiate (represent as or by an instance.) the Prophet object to lets say m 
# P IN PROPHET IS ALWAYS CAPS
m = Prophet()

In [ ]:
m.fit(prophet_df)

In [ ]:
# Forecast the future
future = m.make_future_dataframe(periods = 365)
forecast = m.predict(future)

In [ ]:
forecast

Prophet showed us the trends what are we expecting what are the **Trends**.
* Last row dates to **25th March 2019** Though original data had prices till 25th March 2018 only
* yhat_upper = upper limit.
* yhat_lower = lower limit.

In [ ]:
plt.figure(figsize=(12,8)) # 8 is hight

plt.xticks(np.arange(prophet_df.shape[0])[::5], prophet_df.ds[::5], rotation=90)
plt.scatter(prophet_df['ds'],prophet_df['y'] )

In [ ]:
figure = m.plot( forecast, xlabel= 'Date', ylabel= 'Price')

In [ ]:
m.plot( forecast, xlabel= 'Date', ylabel= 'Price')

In [ ]:
figure = m.plot_components(forecast)

* Why cost of Avocardo increase in month of Oct?
* Why cost of avocardo decreases in january?
* **Are related with harvesting period?**

In [ ]:
# Lets check if Price of Avocardo is raised due to rise in demand.

fig = plt.figure(figsize=(20, 15))

ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

x = df['AveragePrice']
y = df['Date']
z = df['Total Volume']

ax.plot(x, color='blue')
ax.plot(y, color='black')
ax2.plot(y, color='black')
ax2.plot(z, color='green')

'''ax.set_xticks(np.arange(0, len(x)+1, 5))
ax.set_yticks(np.arange(0, max(y), 2))
ax2.set_xticks(np.arange(0, len(x)+1, 25))
ax2.set_yticks(np.arange(0, max(y), 25))'''


### Checking for Chicago

In [ ]:
# Sub set
df_chicago= df[df['region']== 'Chicago']
df_chicago

In [ ]:
# Sort by date
df_chicago= df_chicago.sort_values('Date')
df_chicago

In [ ]:
#
plt.figure(figsize=(10,8))
plt.xticks(np.arange(df_chicago.shape[0])[::5], df_chicago.Date[::5], rotation=90)
plt.plot(df_chicago['Date'], df_chicago['AveragePrice'])

In [ ]:
plt.figure(figsize=(10,8))

# plt.xticks(np.arange(df_chicago.shape[0])[::X], df_chicago.Date[::X], rotation=90)
# X represents interval
plt.xticks(np.arange(df_chicago.shape[0])[::5], df_chicago.Date[::5], rotation=90)

plt.plot(df_chicago['Date'], df_chicago['Total Volume'])

In [ ]:
# fig = plt.figure(figsize=(25, 20))
# Total Volume = Total no of avocado sold

x= df_chicago['Date']
y= df_chicago['AveragePrice']
z= df_chicago['Total Volume']
afig, (ax1, ax2) = plt.subplots(2, figsize=(15,10), sharex=True)
plt.xticks(np.arange(df_chicago.shape[0])[::10], df_chicago.Date[::10], rotation=90)
fig.suptitle('price and volume')
ax1.plot(x, y)
ax2.plot(x, z)


**Here we can observe price of Avocado is inversely proportional to volume of sold avocard.**
* lets compare price of organic avocardo with 

In [ ]:
df_chicago.head(1)

In [ ]:
df_con = df_chicago[df_chicago['type']== 'conventional']
df_con

In [ ]:
# Organic
df_or = df_chicago[df_chicago['type']== 'organic']
df_or

In [ ]:
# Plotting both prices simultaneously of organic and con

X =df_or['Date']
y= df_or['AveragePrice']
z= df_con['AveragePrice']

plt.figure(figsize=(20,8))
plt.xticks(np.arange(df_or.shape[0])[::5], df_or.Date[::5], rotation=90)
plt.plot(X, y, color='r', label='Organic')
plt.plot(X, z, color='g', label='Conventional')
  
# Naming the x-axis, y-axis and the whole graph
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Price of organic vs Conventional")
  
# Adding legend, which helps us recognize the curve according to it's color
plt.legend()

# Placing figsize over here didn't worked but placing it above .plot worked as charme

# To load the display window
plt.show()

# Pro tip ALWAYS PLACE FIGSIZE BEFORE plt.plot.


* Here we can clearly see the cost of Organic Avocado is higher than Conventional avocado.

## Prediction for Chicago 

In [ ]:
df_con= df_con.rename(columns ={'Date':'ds', 'AveragePrice':'y'})
df_con = df_con[['ds', 'y']]
df_con

In [ ]:
m1=Prophet()

In [ ]:
m1.fit(df_con)

In [ ]:
future = m.make_future_dataframe(periods=365)
forecast= m1.predict(future)

In [ ]:
forecast

In [ ]:
figure = m1.plot(forecast, xlabel ='Date', ylabel = 'Price')

We can see there are definatly some Outlier
* n statistics, an outlier is a data point that differs significantly from other observations. An outlier may be due to variability in the measurement or it may indicate experimental error; the latter are sometimes excluded from the data set. An outlier can cause serious problems in statistical analyses.
* **When we predicted the price of avocado on avg it showed it was going down but in case of Chicago it is going down but fall is low compared to previous prediction.**